Import Packages:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql
%matplotlib inline

Take a look at the 2018 hubNashville data by reading it into a DataFrame called hub.

In [2]:
hub = pd.read_csv('../data/hubNashville_2018.csv')

Clean the hub column names to make everything lowercase and eliminate spaces so you can use dot-notation. Make the new names 'request_id', 'status', 'request_type', 'subrequest_type', 'add_subreqest_type', 'opened', 'closed', 'origin', 'zipcode', 'lat', 'lng'.

In [3]:
hub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81166 entries, 0 to 81165
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Request #                   81166 non-null  int64  
 1   Status                      81166 non-null  object 
 2   Request Type                81166 non-null  object 
 3   Subrequest Type             81166 non-null  object 
 4   Additional Subrequest Type  81148 non-null  object 
 5   Date / Time Opened          81166 non-null  object 
 6   Date / Time Closed          80866 non-null  object 
 7   Request Origin              81166 non-null  object 
 8   Zipcode                     81166 non-null  float64
 9   Latitude                    81166 non-null  float64
 10  Longitude                   81166 non-null  float64
dtypes: float64(3), int64(1), object(7)
memory usage: 6.8+ MB


In [5]:
hub.isnull().sum()

Request #                       0
Status                          0
Request Type                    0
Subrequest Type                 0
Additional Subrequest Type     18
Date / Time Opened              0
Date / Time Closed            300
Request Origin                  0
Zipcode                         0
Latitude                        0
Longitude                       0
dtype: int64

In [6]:
hub = hub.rename( columns = {
    'Request #': 'request_id',
    'Status': 'status',
    'Request Type': 'request_type',
    'Subrequest Type': 'subrequest_type',
    'Additional Subrequest Type': 'add_subrequest_type',
    'Date / Time Opened': 'opened',
    'Date / Time Closed': 'closed',
    'Request Origin': 'origin',
    'Zipcode': 'zipcode',
    'Latitude': 'lat',
    'Longitude': 'lng'
})
hub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81166 entries, 0 to 81165
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   request_id           81166 non-null  int64  
 1   status               81166 non-null  object 
 2   request_type         81166 non-null  object 
 3   subrequest_type      81166 non-null  object 
 4   add_subrequest_type  81148 non-null  object 
 5   opened               81166 non-null  object 
 6   closed               80866 non-null  object 
 7   origin               81166 non-null  object 
 8   zipcode              81166 non-null  float64
 9   lat                  81166 non-null  float64
 10  lng                  81166 non-null  float64
dtypes: float64(3), int64(1), object(7)
memory usage: 6.8+ MB


In [ ]:
#hub = hub.loc[~hub.closed.isna()]
#hub.shape

Drop the rows from hub where closed is missing. You should end up with 80,866 rows Then create a new column, resolution_time that calculates how long the request was open. You’ll need to convert opened and closed to pandas datetimes before calculating the time delta.

In [7]:
hub = hub.dropna(subset = ['closed'])
hub['opened'].head()



0    2018-02-12 14:00:20
1    2018-02-12 14:00:21
2    2018-02-02 21:18:15
3    2018-02-23 23:27:07
4    2018-02-08 14:00:04
Name: opened, dtype: object

In [8]:
hub['closed'].head()

0    02/12/2018 03:48:14 PM
1    02/12/2018 03:49:13 PM
2    02/07/2018 05:04:33 PM
3    02/27/2018 09:06:16 PM
4    02/08/2018 03:01:50 PM
Name: closed, dtype: object

In [9]:
hub['opened'] = pd.to_datetime(hub['opened'],
                              format = '%Y-%m-%d %H:%M:%S')
hub['closed'] = pd.to_datetime(hub['closed'],
                              format = '%m/%d/%Y %I:%M:%S %p')
hub['resolution_time'] = hub['closed'] - hub['opened']
hub

,request_id,status,request_type,subrequest_type,add_subrequest_type,opened,closed,origin,zipcode,lat,lng,resolution_time
0,45480,Closed,"Streets, Roads & Sidewalks",Blocking the Right of Way,Blocking the Right of Way,2018-02-12 14:00:20,2018-02-12 15:48:14,Phone,37115.0,36.296917,-86.699162,0 days 01:47:54
1,45482,Closed,"Streets, Roads & Sidewalks",Blocking the Right of Way,Blocking the Right of Way,2018-02-12 14:00:21,2018-02-12 15:49:13,Phone,37076.0,36.215403,-86.588513,0 days 01:48:52
2,43742,Closed,"Trash, Recycling & Litter",Litter,Litter,2018-02-02 21:18:15,2018-02-07 17:04:33,Phone,37076.0,36.187902,-86.607286,4 days 19:46:18
3,49069,Closed,"Streets, Roads & Sidewalks",Blocking the Right of Way,Blocking the Right of Way,2018-02-23 23:27:07,2018-02-27 21:06:16,hubNashville Community,37203.0,36.153761,-86.788745,3 days 21:39:09
4,44743,Closed,"Streets, Roads & Sidewalks",Blocking the Right of Way,Blocking the Right of Way,2018-02-08 14:00:04,2018-02-08 15:01:50,Phone,37210.0,36.154836,-86.763345,0 days 01:01:46
...,...,...,...,...,...,...,...,...,...,...,...,...
81046,38380,Closed,"Trash, Recycling & Litter",Extra Cart Pickup Request,Extra Recycle Cart Pickup,2018-01-11 14:37:00,2018-01-27 11:00:00,Phone,37212.0,36.134800,-86.800880,15 days 20:23:00
81047,39406,Closed,"Streets, Roads & Sidewalks",Snow and Ice Removal,Snow and Ice Removal,2018-01-17 14:29:12,2018-01-17 21:05:07,hubNashville Community,37013.0,36.035460,-86.638371,0 days 06:35:55
81048,39538,Closed,"Streets, Roads & Sidewalks",Snow and Ice Removal,Snow and Ice Removal,2018-01-17 17:05:54,2018-01-17 21:05:13,Phone,37207.0,36.201120,-86.760162,0 days 03:59:19
81072,106123,Closed,Property Violations,Tall Grass and Weeds,Tall Grass and Weeds,2018-10-04 15:01:55,2019-02-27 18:55:35,hubNashville Community,37207.0,36.181081,-86.770350,146 days 03:53:40


Were any requests open for longer than a year? How many? What request type was most commonly open for more than a year? Save the requests that were open for longer than a year to a DataFrame named slow_to_resolve.

In [15]:
slow_to_resolve = hub[hub['resolution_time'] > '365 days']
#alternative: slow_to_resolve = hub.loc[hub.resolution_time > '365 days']
slow_to_resolve.shape

(288, 12)

In [13]:
slow_to_resolve['request_type'].value_counts()

Streets, Roads & Sidewalks    159
Property Violations            73
Public Safety                  14
Other                          14
Electric & Water General       10
Trash, Recycling & Litter       9
Education & Libraries           5
Planning & Zoning               2
Social Services & Housing       2
Name: request_type, dtype: int64

Create a new resolution_time_hours column by dividing the resolution_time column by pd.Timedelta(hours = 1). The code to do this is

In [45]:
hub['resolution_time_hours'] = hub['resolution_time'] / pd.Timedelta(hours = 1)

Look at the distribution of resolution times. What do you notice?

In [46]:
hub['resolution_time'].describe()

count                         80866
mean     14 days 16:05:55.696287685
std      43 days 06:11:36.353865900
min                 0 days 00:00:00
25%          1 days 01:58:26.250000
50%                 3 days 01:16:19
75%                 8 days 11:32:00
max               938 days 02:22:16
Name: resolution_time, dtype: object

Calculate the median resolution time (in hours) by zipcode for requests of type “Streets, Roads & Sidewalks”. We are using median time since the distribution of resolution times is highly skewed. Save the results as a dataframe called streets_median with column names zipcode and median_resolution_time.

In [47]:
streets_median = hub[hub['request_type'] == 'Streets Roads & Sidewalks'].groupby('zipcode')['resolution_time_hours'].median().sort_values()
streets_median = streets_median.reset_index()
streets_median

,zipcode,resolution_time_hours


In [26]:
conn = sql.connect('../data/metro_survey.db')
cur = conn.cursor()

In [28]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = (cur.fetchall())

In [29]:
print(tables)

[('codes',), ('equality_and_access',), ('general_services',), ('info',), ('overall',), ('recreation',), ('safety',), ('transportation',)]


In [30]:
query = '''
SELECT *
FROM safety JOIN info ON safety.Id = info.id;
'''

In [31]:
safety_exp = pd.read_sql(query, conn)
safety_exp.shape

(2171, 29)

In [33]:
safety_exp = safety_exp[['ZIP Code', 'Police - Overall']]
safety_exp.head()

,ZIP Code,Police - Overall
0,37208,Very Dissatisfied
1,37211,Satisfied
2,37208,Neutral
3,37203,Very Satisfied
4,37138,Neutral


In [35]:
safety_total = safety_exp['ZIP Code'].value_counts().reset_index()
safety_total.columns = ['zipcode', 'total_responses']
safety_total.head()

,zipcode,total_responses
0,37207,226
1,37211,190
2,37013,180
3,37221,141
4,37208,121


In [37]:
safety_dissatisfied = safety_exp.loc[safety_exp['Police - Overall'].isin(['Dissatisfied', 'Very Dissatisfied'])]['ZIP Code'].value_counts().reset_index()
safety_dissatisfied.columns = ['zipcode', 'total_dissatisfied']
safety_dissatisfied.head()

,zipcode,total_dissatisfied
0,37207,48
1,37013,36
2,37211,26
3,37218,22
4,37208,22


In [39]:
safety_by_zip = pd.merge(left = safety_dissatisfied, right = safety_total, how = 'right').fillna(0)

In [41]:
safety_by_zip['pct_dissatisfied'] = 100 * safety_by_zip['total_dissatisfied'] / safety_by_zip['total_responses']
safety_by_zip.head()

,zipcode,total_dissatisfied,total_responses,pct_dissatisfied
0,37207,48.0,226,21.238938
1,37211,26.0,190,13.684211
2,37013,36.0,180,20.000000
3,37221,12.0,141,8.510638
4,37208,22.0,121,18.181818


this is as far as I got. The beow is pasted from class, but I am missing a step or two. Revisit the analysis guide to take a look.

In [49]:
#cp_by_zip = police_calls[police_calls['tencode_desc'] == 'Community Policing Activity']['zipcode'].value_counts().reset_index()
#cp_by_zip.columns = ['zipcode', 'community_policing_calls']

NameError: name 'police_calls' is not defined

In [48]:
#safety_by_zip = pd.merge(left = safety_by_zip, right = cp_by_zip, how= 'left')
#safety_by_zip = pd.merge(left = safety_by_zip, right = total_police_calls, how='left').fillna(0)

#safety_by_zip['cp_pct'] = 100 * safety_by_zip['community_policing_calls'] / safety_by_zip['total_calls']

NameError: name 'cp_by_zip' is not defined